{“metadata”:{“kernelspec”:{“language”:“python”,“display_name”:“Python
3”,“name”:“python3”},“language_info”:{“name”:“python”,“version”:“3.10.13”,“mimetype”:“text/x-python”,“codemirror_mode”:{“name”:“ipython”,“version”:3},“pygments_lexer”:“ipython3”,“nbconvert_exporter”:“python”,“file_extension”:“.py”},“kaggle”:{“accelerator”:“nvidiaTeslaT4”,“dataSources”:\[{“sourceId”:8356547,“sourceType”:“datasetVersion”,“datasetId”:3934836}\],“dockerImageVersionId”:30747,“isInternetEnabled”:true,“language”:“python”,“sourceType”:“notebook”,“isGpuEnabled”:true}},“nbformat_minor”:4,“nbformat”:4,“cells”:\[{“cell_type”:“code”,“source”:“#
This Python 3 environment comes with many helpful analytics libraries
installed\# It is defined by the kaggle/python Docker image:
https://github.com/kaggle/docker-python\# For example, here’s several
helpful packages to loadnumpy as np \# linear algebrapandas as pd \#
data processing, CSV file I/O (e.g. pd.read_csv)\# Input data files are
available in the read-only "../input/" directory\# For example, running
this (by clicking run or pressing Shift+Enter) will list all files under
the input directoryosdirname, \_, filenames in
os.walk(‘/kaggle/input’):for filename in
filenames:print(os.path.join(dirname, filename))\# You can write up to
20GB to the current directory (/kaggle/working/) that gets preserved as
output when you create a version using "Save & Run All" \# You can also
write temporary files to /kaggle/temp/, but they won’t be saved outside
of the current
session”,“metadata”:{“\_uuid”:“8f2839f25d086af736a60e9eeb907d3b93b6e0e5”,“\_cell_guid”:“b1076dfc-b9ad-4769-8c92-a6c4dae69d19”,“execution”:{“iopub.status.busy”:“2024-07-18T12:23:24.288986Z”,“iopub.execute_input”:“2024-07-18T12:23:24.289353Z”,“iopub.status.idle”:“2024-07-18T12:23:24.298185Z”,“shell.execute_reply.started”:“2024-07-18T12:23:24.289325Z”,“shell.execute_reply”:“2024-07-18T12:23:24.296831Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“markdown”,“source”:“**Data
Loading and
Preprocessing**”,“metadata”:{}},{“cell_type”:“code”,“source”:“import
pandas as pd\# Load the dataset=
pd.read_csv(‘/kaggle/input/healthcare-dataset/healthcare_dataset.csv’)\#
Check for missing values\_values = df.isnull().sum()("Missing values in
each column:\n",
missing_values)“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:24:42.310227Z”,“iopub.execute_input”:“2024-07-18T12:24:42.311030Z”,“iopub.status.idle”:“2024-07-18T12:24:42.656988Z”,“shell.execute_reply.started”:“2024-07-18T12:24:42.310992Z”,“shell.execute_reply”:“2024-07-18T12:24:42.655968Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“import
matplotlib.pyplot as pltseaborn as sns\# Detect outliers using box
plots\_columns = \[‘Age’, ‘Billing Amount’\]col in
numerical_columns:plt.figure(figsize=(10,
5))sns.boxplot(x=df\[col\])plt.title(f’Box plot of {col}’)plt.show()\#
Handle outliers using the IQR method =
df\[numerical_columns\].quantile(0.25) =
df\[numerical_columns\].quantile(0.75)= Q3 - Q1\_cleaned =
df\[\~((df\[numerical_columns\] \< (Q1 - 1.5 \* IQR)) \|
(df\[numerical_columns\] \> (Q3 + 1.5 \* IQR))).any(axis=1)\]("Data
after handling outliers:\n",
data_cleaned.describe())“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:24:46.013614Z”,“iopub.execute_input”:“2024-07-18T12:24:46.014556Z”,“iopub.status.idle”:“2024-07-18T12:24:46.526217Z”,“shell.execute_reply.started”:“2024-07-18T12:24:46.014520Z”,“shell.execute_reply”:“2024-07-18T12:24:46.525114Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“#
Remove negative values in the Billing Amount column\_cleaned =
data_cleaned\[data_cleaned\[‘Billing Amount’\] \>= 0\]\# Check the
result("Data after removing negative billing amounts:\n",
data_cleaned\[‘Billing
Amount’\].describe())“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:24:53.778618Z”,“iopub.execute_input”:“2024-07-18T12:24:53.779646Z”,“iopub.status.idle”:“2024-07-18T12:24:53.807468Z”,“shell.execute_reply.started”:“2024-07-18T12:24:53.779606Z”,“shell.execute_reply”:“2024-07-18T12:24:53.806400Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“data
=
data_cleaned”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:25:06.418453Z”,“iopub.execute_input”:“2024-07-18T12:25:06.418873Z”,“iopub.status.idle”:“2024-07-18T12:25:06.423704Z”,“shell.execute_reply.started”:“2024-07-18T12:25:06.418838Z”,“shell.execute_reply”:“2024-07-18T12:25:06.422530Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“import
pandas as pdnumpy as np\# Assuming data_cleaned is your cleaned
dataset\# Convert date columns to datetime format=
pd.to_datetime(data\[‘Date of Admission’\])=
pd.to_datetime(data\[‘Discharge Date’\])\# Extract month and year from
admission date= data\[‘Date of Admission’\].dt.month= data\[‘Date of
Admission’\].dt.year\# Define age groups= \[0, 18, 35, 50, 65, 100\]=
\[‘0-18’, ‘19-35’, ‘36-50’, ‘51-65’, ‘66+’\]= pd.cut(data\[‘Age’\],
bins=bins, labels=labels)\# Aggregate data\_data =
data.groupby(\[‘Admission_Year’, ‘Admission_Month’\])\[‘Test
Results’\].apply(lambda x: (x ==
‘Abnormal’).mean()).reset_index()\_data.rename(columns={‘Test Results’:
‘Abnormal_Percentage’}, inplace=True)\# Create a date column for time
series analysis\_data\[‘Date’\] =
pd.to_datetime(grouped_data\[‘Admission_Year’\].astype(str) + ‘-’ +
grouped_data\[‘Admission_Month’\].astype(str))\# Drop unnecessary
columns\_data.drop(columns=\[‘Admission_Year’, ‘Admission_Month’\],
inplace=True)\# Set the date as the index\_data.set_index(‘Date’,
inplace=True)\# Visualize the time seriesmatplotlib.pyplot as
plt.figure(figsize=(12, 6)).plot(grouped_data.index,
grouped_data\[‘Abnormal_Percentage’\]).title(‘Abnormal Test Result
Percentage Over Time’).xlabel(‘Date’).ylabel(‘Abnormal Test Result
Percentage’).grid(True).show()“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:25:09.527542Z”,“iopub.execute_input”:“2024-07-18T12:25:09.528425Z”,“iopub.status.idle”:“2024-07-18T12:25:10.022832Z”,“shell.execute_reply.started”:“2024-07-18T12:25:09.528393Z”,“shell.execute_reply”:“2024-07-18T12:25:10.021646Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“data
=
data.drop_duplicates()”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:25:15.521272Z”,“iopub.execute_input”:“2024-07-18T12:25:15.522053Z”,“iopub.status.idle”:“2024-07-18T12:25:15.623685Z”,“shell.execute_reply.started”:“2024-07-18T12:25:15.522015Z”,“shell.execute_reply”:“2024-07-18T12:25:15.622461Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“data.info()”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:25:17.988743Z”,“iopub.execute_input”:“2024-07-18T12:25:17.989395Z”,“iopub.status.idle”:“2024-07-18T12:25:18.067739Z”,“shell.execute_reply.started”:“2024-07-18T12:25:17.989362Z”,“shell.execute_reply”:“2024-07-18T12:25:18.066708Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“markdown”,“source”:“**Data
Visualization and Stastical
Analysis**”,“metadata”:{}},{“cell_type”:“code”,“source”:“import
matplotlib.pyplot as pltseaborn as sns\# Set up figure with subplots,
axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))\# Plot Age
distribution.histplot(data_cleaned\[‘Age’\], bins=20, kde=True,
ax=axes\[0\], color=‘skyblue’).set_title(‘Distribution of
Age’).set_xlabel(‘Age’)\# Plot Billing Amount
distribution.histplot(data_cleaned\[‘Billing Amount’\], bins=20,
kde=True, ax=axes\[1\], color=‘lightgreen’).set_title(‘Distribution of
Billing Amount’).set_xlabel(‘Billing
Amount’).tight_layout().show()”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:25:53.425789Z”,“iopub.execute_input”:“2024-07-18T12:25:53.426397Z”,“iopub.status.idle”:“2024-07-18T12:25:54.881918Z”,“shell.execute_reply.started”:“2024-07-18T12:25:53.426367Z”,“shell.execute_reply”:“2024-07-18T12:25:54.880924Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“#
Set up figure with subplots, axes = plt.subplots(nrows=3, ncols=1,
figsize=(10, 15))\# Plot Gender distribution.countplot(x=‘Gender’,
data=data_cleaned, ax=axes\[0\]).set_title(‘Distribution of Gender’)\#
Plot Blood Type distribution.countplot(x=‘Blood Type’,
data=data_cleaned, ax=axes\[1\]).set_title(‘Distribution of Blood
Type’)\# Plot Medical Condition distribution (top 10 most frequent
conditions)\_medical_conditions = data_cleaned\[‘Medical
Condition’\].value_counts().nlargest(10).barplot(x=top_medical_conditions.values,
y=top_medical_conditions.index, ax=axes\[2\],
palette=‘viridis’).set_title(‘Top 10 Medical
Conditions’).tight_layout().show()“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:25:58.666057Z”,“iopub.execute_input”:“2024-07-18T12:25:58.666456Z”,“iopub.status.idle”:“2024-07-18T12:25:59.649762Z”,“shell.execute_reply.started”:“2024-07-18T12:25:58.666422Z”,“shell.execute_reply”:“2024-07-18T12:25:59.648697Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“\#
Scatter plot of Age vs Billing Amount.figure(figsize=(8,
6)).scatterplot(x=‘Age’, y=‘Billing Amount’, data=data_cleaned,
hue=‘Test Results’, palette=‘Set2’).title(‘Age vs Billing
Amount’).xlabel(‘Age’).ylabel(‘Billing Amount’).show()\# Pair plot for
selected numerical variables.pairplot(data_cleaned\[\[‘Age’, ‘Billing
Amount’, ‘Room Number’\]\], diag_kind=‘hist’).suptitle(‘Pair Plot of
Numerical Variables’,
y=1.02).show()”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:26:03.289060Z”,“iopub.execute_input”:“2024-07-18T12:26:03.289987Z”,“iopub.status.idle”:“2024-07-18T12:26:11.132016Z”,“shell.execute_reply.started”:“2024-07-18T12:26:03.289951Z”,“shell.execute_reply”:“2024-07-18T12:26:11.130886Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“#
Correlation matrix\_matrix = data_cleaned\[\[‘Age’, ‘Billing Amount’,
‘Room Number’\]\].corr()\# Visualize correlation matrix using a
heatmap.figure(figsize=(8, 6)).heatmap(correlation_matrix, annot=True,
cmap=‘coolwarm’, vmin=-1, vmax=1).title(‘Correlation Matrix of Numerical
Variables’).show()“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T06:42:30.698245Z”,“iopub.execute_input”:“2024-07-18T06:42:30.698670Z”,“iopub.status.idle”:“2024-07-18T06:42:30.988219Z”,“shell.execute_reply.started”:“2024-07-18T06:42:30.698638Z”,“shell.execute_reply”:“2024-07-18T06:42:30.987308Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“from
scipy.stats import chi2_contingency\# Example: Chi-square test for
Gender vs Test Results\_test_results_ct =
pd.crosstab(data_cleaned\[‘Gender’\], data_cleaned\[‘Test Results’\]),
p, dof, expected = chi2_contingency(gender_test_results_ct)("Chi-square
test results for Gender vs Test Results:")(f"Chi-square statistic:
{chi2}")(f"P-value: {p}")(f"Degrees of freedom: {dof}")("Expected
frequencies:\n",
expected)“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T07:58:55.308439Z”,“iopub.execute_input”:“2024-07-18T07:58:55.309001Z”,“iopub.status.idle”:“2024-07-18T07:58:55.350994Z”,“shell.execute_reply.started”:“2024-07-18T07:58:55.308961Z”,“shell.execute_reply”:“2024-07-18T07:58:55.349592Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“from
scipy.stats import ttest_ind, f_oneway\# Example: Two-sample t-test for
Age and Billing Amount between Normal and Abnormal Test Results\_age =
data_cleaned\[data_cleaned\[‘Test Results’\] == ‘Normal’\]\[‘Age’\]\_age
= data_cleaned\[data_cleaned\[‘Test Results’\] ==
‘Abnormal’\]\[‘Age’\]\_stat, p_value = ttest_ind(normal_age,
abnormal_age)("Two-sample t-test results for Age between Normal and
Abnormal Test Results:")(f"T-statistic: {t_stat}")(f"P-value:
{p_value}")\# Example: One-way ANOVA for Billing Amount across different
Admission Types\_results = data_cleaned.groupby(‘Admission
Type’)\[‘Billing Amount’\].apply(list)\_stat, p_value =
f_oneway(\*anova_results)("\nOne-way ANOVA results for Billing Amount
across Admission Types:")(f"F-statistic: {f_stat}")(f"P-value:
{p_value}")“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T06:42:38.821914Z”,“iopub.execute_input”:“2024-07-18T06:42:38.822353Z”,“iopub.status.idle”:“2024-07-18T06:42:38.883963Z”,“shell.execute_reply.started”:“2024-07-18T06:42:38.822315Z”,“shell.execute_reply”:“2024-07-18T06:42:38.882818Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“markdown”,“source”:“**Model
Training, Valuation, Cross validation, Hyperparamater
Tuning**”,“metadata”:{}},{“cell_type”:“code”,“source”:“# Generate lag
featurescreate_lag_features(df, lag=12):for i in range(1, lag +
1):df\[f’lag\_{i}’\] = df\[‘Abnormal_Percentage’\].shift(i)return df\#
Create lag features\_data = create_lag_features(grouped_data)\# Drop
rows with NaN values created by lag
features\_data.dropna(inplace=True)sklearn.preprocessing import
StandardScaler\# Normalize the features= StandardScaler()=
scaler.fit_transform(grouped_data.drop(columns=\[‘Abnormal_Percentage’\]))=
grouped_data\[‘Abnormal_Percentage’\].values\# Reshape data for LSTM
model= X.reshape((X.shape\[0\], X.shape\[1\], 1))\# Split the data into
train and test sets\_size = int(len(grouped_data) \* 0.8), test =
grouped_data\[:train_size\], grouped_data\[train_size:\]\# Split data
into features (X) and target (y)\_train, y_train =
train.drop(columns=\[‘Abnormal_Percentage’\]),
train\[‘Abnormal_Percentage’\]\_test, y_test =
test.drop(columns=\[‘Abnormal_Percentage’\]),
test\[‘Abnormal_Percentage’\]\# Reshape data for LSTM model\_train =
X_train.values.reshape(X_train.shape\[0\], X_train.shape\[1\], 1)\_test
= X_test.values.reshape(X_test.shape\[0\], X_test.shape\[1\],
1)“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:27:20.579973Z”,“iopub.execute_input”:“2024-07-18T12:27:20.580394Z”,“iopub.status.idle”:“2024-07-18T12:27:20.609235Z”,“shell.execute_reply.started”:“2024-07-18T12:27:20.580363Z”,“shell.execute_reply”:“2024-07-18T12:27:20.607958Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“from
keras.models import Sequentialkeras.layers import LSTM, Dense\# Define
LSTM model= Sequential().add(LSTM(50, activation=‘relu’,
input_shape=(X_train.shape\[1\],
1))).add(Dense(1)).compile(optimizer=‘adam’, loss=‘mse’)\# Train the
model.fit(X_train, y_train, epochs=100, batch_size=32,
validation_data=(X_test, y_test),
verbose=1)”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:27:23.068519Z”,“iopub.execute_input”:“2024-07-18T12:27:23.069399Z”,“iopub.status.idle”:“2024-07-18T12:27:47.407090Z”,“shell.execute_reply.started”:“2024-07-18T12:27:23.069363Z”,“shell.execute_reply”:“2024-07-18T12:27:47.406100Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“#
Make predictions\_pred = model.predict(X_test)\# Plot the actual vs
predicted values.figure(figsize=(12, 6)).plot(test.index, y_test,
label=‘Actual’).plot(test.index, y_pred,
label=‘Predicted’).title(‘Actual vs Predicted Abnormal Test Result
Percentage’).xlabel(‘Date’).ylabel(‘Abnormal Test Result
Percentage’).legend().show()\# Evaluate the modelsklearn.metrics import
mean_squared_error, mean_absolute_error, r2_score=
mean_squared_error(y_test, y_pred)= mean_absolute_error(y_test, y_pred)
= r2_score(y_test, y_pred)(f’Mean Squared Error: {mse}‘)(f’Mean Absolute
Error: {mae}’)(f’R-squared:
{r2}’)“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:27:47.408670Z”,“iopub.execute_input”:“2024-07-18T12:27:47.409324Z”,“iopub.status.idle”:“2024-07-18T12:27:48.657281Z”,“shell.execute_reply.started”:“2024-07-18T12:27:47.409292Z”,“shell.execute_reply”:“2024-07-18T12:27:48.656127Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“import
numpy as nppandas as pdsklearn.model_selection import
TimeSeriesSplitsklearn.metrics import mean_squared_error,
mean_absolute_error, r2_scorekeras.models import Sequentialkeras.layers
import LSTM, Densekeras.callbacks import EarlyStopping\# Create
TimeSeriesSplit object= TimeSeriesSplit(n_splits=3)\# Initialize lists
to store the results\_scores = \[\]\_scores = \[\]\_scores = \[\]\#
Prepare the data= grouped_data.drop(columns=\[‘Abnormal_Percentage’\])=
grouped_data\[‘Abnormal_Percentage’\]\# Reshape data for LSTM model=
X.values.reshape(X.shape\[0\], X.shape\[1\], 1)\# Time series
cross-validationtrain_index, test_index in tscv.split(X):X_train, X_test
= X\[train_index\], X\[test_index\]y_train, y_test = y\[train_index\],
y\[test_index\]\# Build the LSTM modelmodel =
Sequential()model.add(LSTM(50, activation=‘relu’,
input_shape=(X_train.shape\[1\], 1),
return_sequences=True))model.add(LSTM(50,
activation=‘relu’))model.add(Dense(1))model.compile(optimizer=‘adam’,
loss=‘mse’)\# Fit the model with early stoppingearly_stopping =
EarlyStopping(monitor=‘val_loss’, patience=10,
restore_best_weights=True)model.fit(X_train, y_train, epochs=200,
batch_size=32, validation_data=(X_test, y_test),
callbacks=\[early_stopping\], verbose=1)\# Make predictionsy_pred =
model.predict(X_test)\# Evaluate the modelmse =
mean_squared_error(y_test, y_pred)mae = mean_absolute_error(y_test,
y_pred)r2 = r2_score(y_test,
y_pred)mse_scores.append(mse)mae_scores.append(mae)r2_scores.append(r2)\#
Calculate average metrics\_mse = np.mean(mse_scores)\_mae =
np.mean(mae_scores)\_r2 = np.mean(r2_scores)(f’Average Mean Squared
Error: {mean_mse}‘)(f’Average Mean Absolute Error:
{mean_mae}’)(f’Average R-squared:
{mean_r2}’)“,”metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:27:57.974578Z”,“iopub.execute_input”:“2024-07-18T12:27:57.974986Z”,“iopub.status.idle”:“2024-07-18T12:28:22.589487Z”,“shell.execute_reply.started”:“2024-07-18T12:27:57.974955Z”,“shell.execute_reply”:“2024-07-18T12:28:22.588394Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]},{“cell_type”:“code”,“source”:“model.save(‘lstm_model.h5’)”,“metadata”:{“execution”:{“iopub.status.busy”:“2024-07-18T12:28:29.648919Z”,“iopub.execute_input”:“2024-07-18T12:28:29.649669Z”,“iopub.status.idle”:“2024-07-18T12:28:29.689336Z”,“shell.execute_reply.started”:“2024-07-18T12:28:29.649617Z”,“shell.execute_reply”:“2024-07-18T12:28:29.688216Z”},“trusted”:true},“execution_count”:null,“outputs”:\[\]}\]}